# 1. Reading & cleaning census survey data

## Introduction

**Purpose:** Restructure data obtained from the US Census Bureau's 5-year [American Community Survey (ACS, 2020 edition)](https://www.census.gov/programs-surveys/acs/news/data-releases/2020/release-schedule.html), which provides socioeconomic data at the census tract level. This workbook focuses on Los Angeles and Orange counties in southern California.

**Contents:** The notebook shows:
* The reading of selected CSV data from the ACS, including basic cleaning of column names. The data read includes basic demographics (e.g.: population; distribution by age, race, and sex), economic factors (e.g.: employment, commuting, and income statistics), and housing data (e.g.: rentals vs. ownership, occupancy rates).
* In total, 6 CSV files are read: one for each broad category of data (3) per county of interest (2).

**Conclusions:** Key conclusions & outputs are:
* The SQLite database `greenspace.db` is created. It includes three tables, one for each category of data obtained from the ACS: `demographics`, `economics`, and `housing`.
* No analysis of the data is included here -- this is in a separate notebook.

## Import required libraries

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
from sqlalchemy import create_engine

## Read & restructure survey data 

Data from the American Community survey is downloadable in CSV format. I conducted minor manual cleanup on the files directly (removing unwanted columns, adding column descriptors, etc). The data is now ready to be read & loaded to a proper SQL database.

### Demographic data

In [4]:
df_demo_la = pd.read_csv("data/acs/la-demographic-data.csv")

In [5]:
df_demo_la["County"] = ["Los Angeles" for i in range(0, len(df_demo_la))]

In [6]:
df_demo_oc = pd.read_csv("data/acs/oc-demographic-data.csv")

In [7]:
df_demo_oc["County"] = ["Orange" for i in range(0, len(df_demo_oc))]

In [8]:
df_demo = pd.concat([df_demo_la, df_demo_oc])
df_demo.head()

,Geography,Geographic Area Name,Total Population,Population (Male),Population (Female),Population (Under 5 years old),Population (5-9 years old),Population (10-14 years old),Population (15-19 years old),Population (20-24 years old),...,Race (one: black or Asian),Race (one: Pacific Islander),Race (one: Other),Race (two+: white and black or African American),Race (two+: white and Native American),Race (two+: white and Asian),Race (two+: black and Native American,Race (Hispanic or Latino),Race (Not Hispanic or Latino),County
0,1400000US06037101110,"Census Tract 1011.10, Los Angeles County, Cali...",3923,1951,1972,199,153,204,209,151,...,406,3,269,63,70,23,0,963,2960,Los Angeles
1,1400000US06037101122,"Census Tract 1011.22, Los Angeles County, Cali...",4119,1971,2148,100,204,248,110,323,...,426,0,36,19,0,111,0,182,3937,Los Angeles
2,1400000US06037101220,"Census Tract 1012.20, Los Angeles County, Cali...",3775,1722,2053,126,161,344,198,209,...,390,0,592,20,40,52,0,1521,2254,Los Angeles
3,1400000US06037101221,"Census Tract 1012.21, Los Angeles County, Cali...",3787,2057,1730,125,170,210,343,261,...,267,0,679,0,0,0,0,1182,2605,Los Angeles
4,1400000US06037101222,"Census Tract 1012.22, Los Angeles County, Cali...",2717,1375,1342,226,179,115,61,220,...,65,0,290,0,29,0,0,1182,1535,Los Angeles


In [9]:
df_demo.dtypes

Geography                                           object
Geographic Area Name                                object
Total Population                                     int64
Population (Male)                                    int64
Population (Female)                                  int64
Population (Under 5 years old)                       int64
Population (5-9 years old)                           int64
Population (10-14 years old)                         int64
Population (15-19 years old)                         int64
Population (20-24 years old)                         int64
Population (25-34 years old)                         int64
Population (35-44 years old)                         int64
Population (45-54 years old)                         int64
Population (55-59 years old)                         int64
Population (60-64 years old)                         int64
Population (65-74 years old)                         int64
Population (75-84 years old)                         int

Most of these column names aren't formatted properly for SQL usage. Remove parentheses and spaces, replacing both with a dash.

In [10]:
features = list(df_demo.columns)

In [11]:
features_clean = [feature.replace(" ", "_").replace("(", "").replace(")", "").replace(":", "").replace("-", "to") for feature in features]
features_clean

['Geography',
 'Geographic_Area_Name',
 'Total_Population',
 'Population_Male',
 'Population_Female',
 'Population_Under_5_years_old',
 'Population_5to9_years_old',
 'Population_10to14_years_old',
 'Population_15to19_years_old',
 'Population_20to24_years_old',
 'Population_25to34_years_old',
 'Population_35to44_years_old',
 'Population_45to54_years_old',
 'Population_55to59_years_old',
 'Population_60to64_years_old',
 'Population_65to74_years_old',
 'Population_75to84_years_old',
 'Population_85+_years_old',
 'Race_One_Race',
 'Race_Two_or_More_Races',
 'Race_one_white',
 'Race_one_black_or_African_American',
 'Race_one_native_American',
 'Race_one_black_or_Asian',
 'Race_one_Pacific_Islander',
 'Race_one_Other',
 'Race_two+_white_and_black_or_African_American',
 'Race_two+_white_and_Native_American',
 'Race_two+_white_and_Asian',
 'Race_two+_black_and_Native_American',
 'Race_Hispanic_or_Latino',
 'Race_Not_Hispanic_or_Latino',
 'County']

In [12]:
df_demo.columns = features_clean

Finally, reindex the dataframe on the tract's unique identifying FIPS code.

In [13]:
geographies = list(df_demo["Geography"])
tract_fips = [geography.split("US")[1] for geography in geographies]
df_demo["Tract_FIPS"] = tract_fips
df_demo = df_demo.set_index("Tract_FIPS")

In [14]:
df_demo.head()

,Geography,Geographic_Area_Name,Total_Population,Population_Male,Population_Female,Population_Under_5_years_old,Population_5to9_years_old,Population_10to14_years_old,Population_15to19_years_old,Population_20to24_years_old,...,Race_one_black_or_Asian,Race_one_Pacific_Islander,Race_one_Other,Race_two+_white_and_black_or_African_American,Race_two+_white_and_Native_American,Race_two+_white_and_Asian,Race_two+_black_and_Native_American,Race_Hispanic_or_Latino,Race_Not_Hispanic_or_Latino,County
Tract_FIPS,,,,,,,,,,,,,,,,,,,,,
06037101110,1400000US06037101110,"Census Tract 1011.10, Los Angeles County, Cali...",3923,1951,1972,199,153,204,209,151,...,406,3,269,63,70,23,0,963,2960,Los Angeles
06037101122,1400000US06037101122,"Census Tract 1011.22, Los Angeles County, Cali...",4119,1971,2148,100,204,248,110,323,...,426,0,36,19,0,111,0,182,3937,Los Angeles
06037101220,1400000US06037101220,"Census Tract 1012.20, Los Angeles County, Cali...",3775,1722,2053,126,161,344,198,209,...,390,0,592,20,40,52,0,1521,2254,Los Angeles
06037101221,1400000US06037101221,"Census Tract 1012.21, Los Angeles County, Cali...",3787,2057,1730,125,170,210,343,261,...,267,0,679,0,0,0,0,1182,2605,Los Angeles
06037101222,1400000US06037101222,"Census Tract 1012.22, Los Angeles County, Cali...",2717,1375,1342,226,179,115,61,220,...,65,0,290,0,29,0,0,1182,1535,Los Angeles


Write the new data to the `demographics` table in the database.

In [15]:
with create_engine("sqlite:///greenspace.db").connect() as db:
    df_demo.to_sql("demographics", db)

### Economic data

In [16]:
df_econ_la = pd.read_csv("data/acs/la-economic-data.csv")

In [17]:
df_econ_la["County"] = ["Los Angeles" for i in range(0, len(df_econ_la))]

In [18]:
df_econ_oc = pd.read_csv("data/acs/oc-economic-data.csv")

In [19]:
df_econ_oc["County"] = ["Orange" for i in range(0, len(df_econ_oc))]

In [20]:
df_econ = pd.concat([df_econ_la, df_econ_oc])
df_econ.head()

,Geography,Geographic Area Name,"Employed population (16+ years, all)","Employed population (16+ years, female)",Workers who commute to work (total),Workers who commute to work (drive alone),Workers who commute to work (carpool),Workers who commute to work (public transit),Workers who commute to work (walk),Workers who commute to work (other),Workers who commute to work (work from home),Mean travel time to work (minutes),Median household income (dollars),Mean household income (dollars),County
0,1400000US06037101110,"Census Tract 1011.10, Los Angeles County, Cali...",3356,1715,1953,1586,223,60,0,18,66,32.4,74625,82334,Los Angeles
1,1400000US06037101122,"Census Tract 1011.22, Los Angeles County, Cali...",3548,1809,1997,1722,103,8,0,34,130,40.7,93125,120908,Los Angeles
2,1400000US06037101220,"Census Tract 1012.20, Los Angeles County, Cali...",3123,1678,1745,1372,83,105,16,67,102,29,55682,78330,Los Angeles
3,1400000US06037101221,"Census Tract 1012.21, Los Angeles County, Cali...",3216,1426,1631,1247,202,56,43,72,11,36.4,46274,55706,Los Angeles
4,1400000US06037101222,"Census Tract 1012.22, Los Angeles County, Cali...",2197,1150,1165,940,116,0,30,14,65,24.5,30016,42809,Los Angeles


In [21]:
df_econ.dtypes

Geography                                       object
Geographic Area Name                            object
Employed population (16+ years, all)             int64
Employed population (16+ years, female)          int64
Workers who commute to work (total)              int64
Workers who commute to work (drive alone)        int64
Workers who commute to work (carpool)            int64
Workers who commute to work (public transit)     int64
Workers who commute to work (walk)               int64
Workers who commute to work (other)              int64
Workers who commute to work (work from home)     int64
Mean travel time to work (minutes)              object
Median household income (dollars)               object
Mean household income (dollars)                 object
County                                          object
dtype: object

A few of these columns are stored as strings rather than numeric values currently. Let's fix that:

In [22]:
str_cols = ["Mean travel time to work (minutes)", "Median household income (dollars)", "Mean household income (dollars)"]

In [23]:
for column in str_cols:
    df_econ[column] = pd.to_numeric(df_econ[column],
                                    errors = "coerce")

Let's double check that not too many values were unreadable when using the `coerce` argument:

In [24]:
df_econ.isna().sum() / len(df_econ) > 0.1

Geography                                       False
Geographic Area Name                            False
Employed population (16+ years, all)            False
Employed population (16+ years, female)         False
Workers who commute to work (total)             False
Workers who commute to work (drive alone)       False
Workers who commute to work (carpool)           False
Workers who commute to work (public transit)    False
Workers who commute to work (walk)              False
Workers who commute to work (other)             False
Workers who commute to work (work from home)    False
Mean travel time to work (minutes)              False
Median household income (dollars)               False
Mean household income (dollars)                 False
County                                          False
dtype: bool

That's workable -- less than 10% of each column is a null value after coercing the data into a numeric type.

Further, we also need to fix the formatting of these column names:

In [25]:
features = list(df_econ.columns)

In [26]:
features_clean = [feature.replace(" ", "_").replace("(", "").replace(")", "").replace(":", "").replace("-", "to") for feature in features]
features_clean

['Geography',
 'Geographic_Area_Name',
 'Employed_population_16+_years,_all',
 'Employed_population_16+_years,_female',
 'Workers_who_commute_to_work_total',
 'Workers_who_commute_to_work_drive_alone',
 'Workers_who_commute_to_work_carpool',
 'Workers_who_commute_to_work_public_transit',
 'Workers_who_commute_to_work_walk',
 'Workers_who_commute_to_work_other',
 'Workers_who_commute_to_work_work_from_home',
 'Mean_travel_time_to_work_minutes',
 'Median_household_income_dollars',
 'Mean_household_income_dollars',
 'County']

In [27]:
df_econ.columns = features_clean

Reindex the dataframe on the tract's FIPS code:

In [28]:
geographies = list(df_econ["Geography"])
tract_fips = [geography.split("US")[1] for geography in geographies]
df_econ["Tract_FIPS"] = tract_fips
df_econ = df_econ.set_index("Tract_FIPS")

In [29]:
df_econ.head()

,Geography,Geographic_Area_Name,"Employed_population_16+_years,_all","Employed_population_16+_years,_female",Workers_who_commute_to_work_total,Workers_who_commute_to_work_drive_alone,Workers_who_commute_to_work_carpool,Workers_who_commute_to_work_public_transit,Workers_who_commute_to_work_walk,Workers_who_commute_to_work_other,Workers_who_commute_to_work_work_from_home,Mean_travel_time_to_work_minutes,Median_household_income_dollars,Mean_household_income_dollars,County
Tract_FIPS,,,,,,,,,,,,,,,
06037101110,1400000US06037101110,"Census Tract 1011.10, Los Angeles County, Cali...",3356,1715,1953,1586,223,60,0,18,66,32.4,74625.0,82334.0,Los Angeles
06037101122,1400000US06037101122,"Census Tract 1011.22, Los Angeles County, Cali...",3548,1809,1997,1722,103,8,0,34,130,40.7,93125.0,120908.0,Los Angeles
06037101220,1400000US06037101220,"Census Tract 1012.20, Los Angeles County, Cali...",3123,1678,1745,1372,83,105,16,67,102,29.0,55682.0,78330.0,Los Angeles
06037101221,1400000US06037101221,"Census Tract 1012.21, Los Angeles County, Cali...",3216,1426,1631,1247,202,56,43,72,11,36.4,46274.0,55706.0,Los Angeles
06037101222,1400000US06037101222,"Census Tract 1012.22, Los Angeles County, Cali...",2197,1150,1165,940,116,0,30,14,65,24.5,30016.0,42809.0,Los Angeles


Finally, write the new data to the `economics` table in the database:

In [30]:
with create_engine("sqlite:///greenspace.db").connect() as db:
    df_econ.to_sql("economics", db)

### Housing data

In [31]:
df_housing_la = pd.read_csv("data/acs/la-housing-data.csv")

In [32]:
df_housing_la["County"] = ["Los Angeles" for i in range(0, len(df_housing_la))]

In [33]:
df_housing_oc = pd.read_csv("data/acs/oc-housing-data.csv")

In [34]:
df_housing_oc["County"] = ["Orange" for i in range(0, len(df_housing_oc))]

In [35]:
df_housing = pd.concat([df_housing_la, df_housing_oc])
df_housing.head()

,Geography,Geographic Area Name,Housing units (total),Housing units (occupied),Housing units (vacant),Vacancy rate (homeowner),Vacancy rate (rental),"Housing units in structure (1, detached)","Housing units in structure (1, attached)",Housing units in structure (2),...,"Monthly rent, less than 500 dollars, count","Monthly rent, 500 to 1k dollars, count","Monthly rent, 1k to 1.5k dollars, count","Monthly rent, 1.5k to 2k dollars, count","Monthly rent, 2k to 2.5k dollars, count","Monthly rent, 2.5k to 3k dollars, count","Monthly rent, more than 3k dollars, count","Monthly rent, median dollars","Monthly rent, no rent paid",County
0,1400000US06037101110,"Census Tract 1011.10, Los Angeles County, Cali...",1629,1505,124,0,0,1069,63,81,...,23,60,127,212,140,47,0,1723,18,Los Angeles
1,1400000US06037101122,"Census Tract 1011.22, Los Angeles County, Cali...",1406,1341,65,0,0,1406,0,0,...,0,0,28,72,43,33,68,2256,92,Los Angeles
2,1400000US06037101220,"Census Tract 1012.20, Los Angeles County, Cali...",1462,1430,32,0,0,817,156,4,...,74,124,355,237,13,0,16,1291,0,Los Angeles
3,1400000US06037101221,"Census Tract 1012.21, Los Angeles County, Cali...",1567,1513,54,0,2.2,310,76,12,...,23,53,653,264,82,37,32,1416,61,Los Angeles
4,1400000US06037101222,"Census Tract 1012.22, Los Angeles County, Cali...",986,969,17,0,0,74,42,44,...,0,48,483,216,98,0,0,1330,14,Los Angeles


In [36]:
df_housing.dtypes

Geography                                                   object
Geographic Area Name                                        object
Housing units (total)                                        int64
Housing units (occupied)                                     int64
Housing units (vacant)                                       int64
Vacancy rate (homeowner)                                    object
Vacancy rate (rental)                                       object
Housing units in structure (1, detached)                     int64
Housing units in structure (1, attached)                     int64
Housing units in structure  (2)                              int64
Housing units in structure (3-4)                             int64
Housing units in structure (5-9)                             int64
Housing units in structure (10-19)                           int64
Housing units in structure (20+)                             int64
Housing units in structure (mobile home)                     i

A few of the columns were read as strings rather than numeric values here, too. Fixing that:

In [37]:
str_cols = ["Vacancy rate (homeowner)",
            "Vacancy rate (rental)",
            "Housing, median number rooms",
            "Owner-occupied Unit value, median dollars",
            "Monthly rent, median dollars"]

In [38]:
for column in str_cols:
    df_housing[column] = pd.to_numeric(df_housing[column],
                                       errors = "coerce")

Let's make sure the `coerce` argument didn't result in too many non-numeric values being dropped:

In [39]:
(df_housing.isna().sum() / len(df_housing)) > 0.10

Geography                                                   False
Geographic Area Name                                        False
Housing units (total)                                       False
Housing units (occupied)                                    False
Housing units (vacant)                                      False
Vacancy rate (homeowner)                                    False
Vacancy rate (rental)                                       False
Housing units in structure (1, detached)                    False
Housing units in structure (1, attached)                    False
Housing units in structure  (2)                             False
Housing units in structure (3-4)                            False
Housing units in structure (5-9)                            False
Housing units in structure (10-19)                          False
Housing units in structure (20+)                            False
Housing units in structure (mobile home)                    False
Housing un

That's good enough -- > 90% of original values were retained after coercing all the columns to a numeric data type.

Further, we also need to fix the formatting of these column names:

In [40]:
features = list(df_housing.columns)

In [41]:
features_clean = [feature.replace(" ", "_").replace("(", "").replace(")", "").replace(":", "").replace("-", "to") for feature in features]
features_clean

['Geography',
 'Geographic_Area_Name',
 'Housing_units_total',
 'Housing_units_occupied',
 'Housing_units_vacant',
 'Vacancy_rate_homeowner',
 'Vacancy_rate_rental',
 'Housing_units_in_structure_1,_detached',
 'Housing_units_in_structure_1,_attached',
 'Housing_units_in_structure__2',
 'Housing_units_in_structure_3to4',
 'Housing_units_in_structure_5to9',
 'Housing_units_in_structure_10to19',
 'Housing_units_in_structure_20+',
 'Housing_units_in_structure_mobile_home',
 'Housing_units_in_structure_RV,_boat,_etc',
 'Year_of_housing_structures_built_after_2014',
 'Year_of_housing_structures_built_2010to2013',
 'Year_of_housing_structures_built_2000to2009',
 'Year_of_housing_structures_built_1990to1999',
 'Year_of_housing_structures_built_1980to1989',
 'Year_of_housing_structures_built_1970to1979',
 'Year_of_housing_structures_built_1960to1969',
 'Year_of_housing_structures_built_1950to1959',
 'Year_of_housing_structures_built_1940to1949',
 'Year_of_housing_structures_built_1939_or_before

In [42]:
df_housing.columns = features_clean

Reindex the dataframe on the tract's unique identifying FIPS code.

In [43]:
geographies = list(df_housing["Geography"])
tract_fips = [geography.split("US")[1] for geography in geographies]
df_housing["Tract_FIPS"] = tract_fips
df_housing = df_housing.set_index("Tract_FIPS")

In [44]:
df_housing.head()

,Geography,Geographic_Area_Name,Housing_units_total,Housing_units_occupied,Housing_units_vacant,Vacancy_rate_homeowner,Vacancy_rate_rental,"Housing_units_in_structure_1,_detached","Housing_units_in_structure_1,_attached",Housing_units_in_structure__2,...,"Monthly_rent,_less_than_500_dollars,_count","Monthly_rent,_500_to_1k_dollars,_count","Monthly_rent,_1k_to_1.5k_dollars,_count","Monthly_rent,_1.5k_to_2k_dollars,_count","Monthly_rent,_2k_to_2.5k_dollars,_count","Monthly_rent,_2.5k_to_3k_dollars,_count","Monthly_rent,_more_than_3k_dollars,_count","Monthly_rent,_median_dollars","Monthly_rent,_no_rent_paid",County
Tract_FIPS,,,,,,,,,,,,,,,,,,,,,
06037101110,1400000US06037101110,"Census Tract 1011.10, Los Angeles County, Cali...",1629,1505,124,0.0,0.0,1069,63,81,...,23,60,127,212,140,47,0,1723.0,18,Los Angeles
06037101122,1400000US06037101122,"Census Tract 1011.22, Los Angeles County, Cali...",1406,1341,65,0.0,0.0,1406,0,0,...,0,0,28,72,43,33,68,2256.0,92,Los Angeles
06037101220,1400000US06037101220,"Census Tract 1012.20, Los Angeles County, Cali...",1462,1430,32,0.0,0.0,817,156,4,...,74,124,355,237,13,0,16,1291.0,0,Los Angeles
06037101221,1400000US06037101221,"Census Tract 1012.21, Los Angeles County, Cali...",1567,1513,54,0.0,2.2,310,76,12,...,23,53,653,264,82,37,32,1416.0,61,Los Angeles
06037101222,1400000US06037101222,"Census Tract 1012.22, Los Angeles County, Cali...",986,969,17,0.0,0.0,74,42,44,...,0,48,483,216,98,0,0,1330.0,14,Los Angeles


Finally, write the new data to the `housing` table in the database:

In [45]:
with create_engine("sqlite:///greenspace.db").connect() as db:
    df_housing.to_sql("housing", db)